# Text Representation

Please, note that this notebook is intended to be run in Google Colab.

In [1]:
# Mount drive and define path to the data folder (from your Google Drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Exercise 1

Load the [metadata file](https://nijianmo.github.io/amazon/index.html) and discard any item that was not rated by our subset of users (nor in training or test sets). Apply preprocessing (stemming and stopwords removal) to clean up the text from the "title". Report the vocabulary size before and after the preprocessing.

In [2]:
import json
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('../')
import pickle

In [3]:
datapath = 'drive/MyDrive/data/'
train_file = 'traindf.csv'
test_file = 'testdf.csv'
meta_file = 'meta_All_Beauty.json.gz'
# Load TRAIN and TEST sets 
traindf = pd.read_csv(datapath+train_file,index_col=0)
testdf = pd.read_csv(datapath+test_file,index_col=0)
# Load the METADATA (ITEMS)
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(datapath+meta_file)
# Discard duplicates
norepeat_df = df.drop_duplicates(subset='asin',ignore_index=True)
print(len(norepeat_df))
# Discard items that weren't rated by our subset of users
train_items = set(traindf['asin'].drop_duplicates())
test_items = set(testdf['asin'].drop_duplicates())
meta_items = set(df['asin'].drop_duplicates())
item_in_train_or_test = (train_items | test_items) & meta_items
meta_df = norepeat_df.loc[norepeat_df['asin'].isin(item_in_train_or_test)]


32488


In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import string

# <YOUR CODE HERE>

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
meta_df.reset_index(drop=True,inplace=True)

In [6]:
count_before = 0
count_after = 0
before_list = []
after_list = []
new_title_list = []
raw_title_list = []
for i in range(len(meta_df)):
  sentence = meta_df.loc[i,'title']
  raw_title_list.append(sentence)
  before_list.extend(word_tokenize(sentence))
  stemming_sentence = ps.stem(sentence)
  word_tokens = word_tokenize(stemming_sentence)
  alpha = [w.lower() for w in word_tokens if w.isalpha()]
  filtered_sentence = [w for w in alpha if not w in stop_words]
  after_list.extend(filtered_sentence)
  new_title_list.append(filtered_sentence)
print(len(np.unique(np.array(before_list))),len(np.unique(np.array(after_list))))

545 399


In [12]:
meta_df.to_csv(datapath+'meta_df.csv')

# Exercise 2

Representation in vector spaces.

## 2.1

Represent all the products from Exercise 1 in a TF-IDF space. Interpret the meaning of the TF-IDF matrix dimensions.

Tip: You may use the library [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

for i in range(len(new_title_list)):
  new_title_list[i]=" ".join(new_title_list[i])
matrix_X = tfidf_vectorizer.fit_transform(new_title_list)
print(matrix_X.shape)

(84, 396)


## 2.2

Compute and the cosine similarity between products with asin 'B000FI4S1E', 'B000LIBUBY' and 'B000W0C07Y'. Take a look at their features to see whether results make sense with their characteristics. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
matrix_X_df = pd.DataFrame(matrix_X.toarray(),index=meta_df['asin'])
cosine_similarity(matrix_X_df.loc[['B000FI4S1E','B000LIBUBY','B000W0C07Y']])

array([[1.        , 0.03587161, 0.02879751],
       [0.03587161, 1.        , 0.3599383 ],
       [0.02879751, 0.3599383 , 1.        ]])

In [ ]:
matrix_X_df.to_csv(datapath+'item_feature.csv')

# Exercise 3

Representation in vector spaces with contextual Word Embeddings.

## 3.1.

Represent all the products from Exercise 1 in a vector space using embeddings from a pre-trained BERT model. The final embedding of a product should be the average of the word embeddings from all the words in the 'title'. What is the vocabulary size of the model? What are the dimensions of the last hidden state?

Tip: you may install the transformers library and use their pretrained [BERT model uncased](https://huggingface.co/bert-base-uncased).

In [ ]:
# LOAD TRANSFORMER
"""
If you plan on using a pretrained model, it’s important to use the associated 
pretrained tokenizer: it will split the text you give it in tokens the same way
for the pretraining corpus, and it will use the same correspondence
token to index (that we usually call a vocab) as during pretraining.
"""

!pip install transformers
import torch
import transformers
assert transformers.__version__ > '4.0.0'

from transformers import BertModel, BertTokenizerFast,BertConfig

# set-up environment
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


modelname = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(modelname)
model = BertModel.from_pretrained(modelname).to(DEVICE)

# Print out the vocabulary size
configuration = model.config
print(configuration.vocab_size,configuration.hidden_size)

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 6.5 MB 30.6 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Using device: cuda:0


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


30522 768


In [ ]:
# REPRESENT PRODUCTS IN A VECTOR SPACE


def batch_encoding(sentences):
    # Since we're using padding, we need to provide the attention masks to our
    # model. Otherwise it doesn't know which tokens it should not attend to. 
    inputs = tokenizer(sentences, padding=True,return_tensors='pt').to(DEVICE)
    # print(inputs) # Look at the padding and attention_mask

    outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state

    return inputs, last_hidden_states
  
encoded_inputs, title_last_hidden_states = batch_encoding(raw_title_list)

"""
Note that the control token [CLS] has been added 
at the beginning of each sentence, and [SEP] at the end. 
"""

# Now, let's mask out the padding tokens and compute the embedding vector of each product

print(title_last_hidden_states.shape)
item_vectors = []
for i in range(title_last_hidden_states.shape[0]):
  word_vectors = torch.zeros(768).cuda()
  for j in range(title_last_hidden_states.shape[1]):
    if encoded_inputs.attention_mask[i,j]==1:
      word_vectors += title_last_hidden_states[i][j]
      word_num = encoded_inputs.attention_mask[i].sum()
  item_vectors.append((word_vectors/word_num))

torch.Size([84, 52, 768])


## 3.2.

Compute and the cosine similarity between products with asin 'B000FI4S1E', 'B000LIBUBY' and 'B000W0C07Y'.

In [ ]:
three_index = meta_df[meta_df['asin'].isin(['B000FI4S1E','B000LIBUBY','B000W0C07Y'])].index

In [ ]:
a = item_vectors[three_index[0]].cpu().detach().numpy()
b = item_vectors[three_index[1]].cpu().detach().numpy()
c = item_vectors[three_index[2]].cpu().detach().numpy()
three_matrix = np.row_stack((a,b,c))
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(three_matrix)

array([[0.99999994, 0.7335933 , 0.6593504 ],
       [0.7335933 , 1.0000004 , 0.74751186],
       [0.6593504 , 0.74751186, 0.9999998 ]], dtype=float32)